In [4]:
!pip install opencv-python

In [1]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('test.jpg')
# Create a resizable window
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
# Resize the window (you can adjust the size as needed)
cv2.resizeWindow('image', 800, 600)  # Adjust the width and height
cv2.imshow('image',image)
cv2.waitKey(0)

-1

In [2]:

# Convert the image from BGR to HSV color space
img_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define color ranges (HSV format) for red, green, blue, and yellow
color_ranges = {
    "Red": ([0, 100, 100], [10, 255, 255]),
    "Green": ([36, 100, 100], [86, 255, 255]),
    "Blue": ([100, 100, 100], [140, 255, 255]),
    "Yellow": ([20, 100, 100], [30, 255, 255])
}

# Iterate over the color ranges to create masks and detect shapes
for color_name, (lower, upper) in color_ranges.items():
    # Convert to NumPy arrays for the lower and upper bounds
    lower_bound = np.array(lower, dtype="uint8")
    upper_bound = np.array(upper, dtype="uint8")

    # Create a mask for the current color
    mask = cv2.inRange(img_HSV, lower_bound, upper_bound)
    
    # blur the mask to reduce noise
    mask = cv2.GaussianBlur(mask, (5, 5), 0) 

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.011 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Draw the contour and approximate polygon
        cv2.drawContours(image, [approx], 0, (0, 0, 0), 5)
        
        # Find the center of the shape to label it
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        
            # Label the color of the shape
            cv2.putText(image, color_name, (cx - 50, cy + 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)
        
        # Shape detection based on the number of vertices
        x, y, w, h = cv2.boundingRect(approx)
        aspectRatio = float(w) / h
        
        if len(approx) == 3:
            cv2.putText(image, "Triangle", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 0), 2)
        elif len(approx) == 4:
            if 0.95 <= aspectRatio <= 1.05:
                cv2.putText(image, "Square", (x, y - 20), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 0), 2)
            else:
                cv2.putText(image, "Rectangle", (x+10, y - 10), cv2.FONT_HERSHEY_COMPLEX, 1.3, (0, 0, 0), 2)
        else:
            cv2.putText(image, "Circle", (x + 70, y - 10), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 0), 2)

# Display the output image
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 800, 600)  # Adjust the width and height
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
